<a href="https://colab.research.google.com/github/ItsShi/Mono-Depth/blob/main/ASF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download

In [8]:
!rm -rf /content/asdf
!git clone https://github.com/ItsShi/asdf.git

for opt in ["train", "val", "test"]:
    f = open("/content/asdf/splits/endovis/{}_files.txt".format(opt), "r+")
    # f = open("/content/{}_files_original.txt".format(opt), "r+")
    new_file = []
    for line in f:
      folder, frame = line.split()[0], line.split()[1]
      if int(frame)<20 and int(frame)>0:
        if folder == "dataset3/keyframe1" or folder == "dataset3/keyframe3" or folder == "dataset3/keyframe4":
          new_file.append(line)
    print(len(new_file))
    with open("{}_files.txt".format(opt), "w+") as f:
      for i in new_file:
        f.write(i)

    !cp -f /content/{opt}_files.txt /content/asdf/splits/endovis/{opt}_files.txt

Cloning into 'asdf'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 45 (delta 11), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (45/45), 282.78 KiB | 4.28 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
!pip -q install timm tensorboardX==1.4 torchvision==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 917.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.


In [3]:
import gdown

perfect_gt_url = 'https://drive.google.com/uc?id=1GHX4ML7kUP4xFZSi56VtpZSxZdwHa2oO'
gdown.download(perfect_gt_url,'perfect_gt.zip',quiet=True)
!unzip -q perfect_gt.zip

import gdown

!mkdir videos
d3k1_url = 'https://drive.google.com/uc?id=17k1-CHptG_XTXVX8qp7yK_zadH54vVnY'
gdown.download(d3k1_url,'videos/d3k1_rgb.mp4',quiet=True)
# frame=  329 fps= 23 q=24.8 Lsize=N/A time=00:00:13.16 bitrate=N/A speed=0.904x

!pip install ffmpeg

video_name = "d3k1"
!ffmpeg -i /content/videos/{video_name}_rgb.mp4 -filter:v "crop=1280:1024:0:0" /content/videos/{video_name}_crop_rgb.mp4
!ffmpeg -i /content/videos/{video_name}_crop_rgb.mp4 %6d.jpg
!mkdir -p /content/frames/{video_name}
!mv *.jpg /content/frames/{video_name}

import PIL.Image as pil
im = pil.open("/content/frames/{}/000001.jpg".format(video_name))
print(im.size)

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=620a8267cc6c96280301ade0bc0f1913f3e691847266814a9159d4376f50b9d2
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --e

In [10]:
# change image path in datasets/scared_dataset.py
def replace_text_in_file(search_path, replace_path, search_text, replace_text):
    with open(search_path, 'r') as file: content = file.read()
    for i in range(len(search_text)):
      modified_content = content.replace(search_text[i], replace_text[i])
      content = modified_content
    with open(replace_path, 'w') as file: file.write(modified_content)

# change splits/endovis/_files.txt
for opt in ["train", "val", "test"]:
    f = open("/content/asdf/splits/endovis/{}_files.txt".format(opt), "r+")
    # f = open("/content/{}_files_original.txt".format(opt), "r+")
    new_file = []
    for line in f:
      folder, frame = line.split()[0], line.split()[1]
      if int(frame)<20 and int(frame)>0:
        if folder == "dataset3/keyframe1" or folder == "dataset3/keyframe3" or folder == "dataset3/keyframe4":
          new_file.append(line)
    print(len(new_file))
    with open("{}_files.txt".format(opt), "w+") as f:
      for i in new_file:
        f.write(i)

    !cp -f /content/{opt}_files.txt /content/asdf/splits/endovis/{opt}_files.txt

16
16
2


In [11]:
!python asdf/train_end_to_end.py --no_cuda --data_path frames/d3k1 --log_dir asdf  --num_epochs 1 --batch_size 1 # --scales 0 1 2  #--load_weights_folder womin/mdp/models/weights_19  --models_to_load "encoder" "depth" "pose_encoder" "pose"

Training model named:
   mdp
Models and tensorboard events files are saved to:
   asdf
Training is using:
   cpu
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Using split:
   endovis
There are 16 training items and 16 validation items

Training..
Processing batch..
input:  torch.Size([1, 3, 256, 320])
features:  5 torch.Size([1, 64, 128, 160]) t

In [4]:
url = 'https://drive.google.com/uc?id=1zGXr3fTI-zdm_A9gViQrzzJ2qFvbhGN-'
gdown.download(url,'weights_12.zip',quiet=True)
!unzip -q weights_12.zip

In [15]:
# test sequence1: d5k4 dataset5/keyframe4	1	l
# test sequence2 d3k4
# seq3 d3k1 frames/d3k1/000001.jpg

# import gdown
# import shutil

# video_name = "d3k1"
# camera_url = 'https://drive.google.com/uc?id=1_G5h2CUXqdREPj8dB7DorVDQHxP4oLvF'
# gdown.download(camera_url,'SCARED_camera.zip',quiet=True)
# !unzip -q /content/SCARED_camera.zip
# shutil.unpack_archive("SCARED_camera/{}_frame_data.tar.gz".format(video_name), "{}_json".format(video_name))

# !python thesis/export_gt_pose.py --data_path d3k1_json --split endovis

Saving to endovis


In [7]:
import os

with open("{}_sequence.txt".format(video_name), "w+") as f:
    for i in range(2, len(os.listdir('d3k1_json')) + 1):
        f.write("dataset{}/keyframe{}\t{}\tl\n".format(video_name[1], video_name[-1],i))

!cp d3k1_sequence.txt thesis/splits/endovis/

In [ ]:
!python thesis/evaluate_pose.py --data_path frames/d3k1 --eval_split endovis --load_weights_folder weights_12

In [ ]:
!python thesis/evaluate_pose.py --data_path frames/d3k1 --eval_split endovis --load_weights_folder weights_12


# SAM

# evaluate

# lite-mono only use 3 scales

In [ ]:
!python thesis/train_end_to_end.py --no_cuda --scales 0 1 2 --data_path frames/d3k1 --log_dir sam  --batch_size 3 #--load_weights_folder womin/mdp/models/weights_19  --models_to_load "encoder" "depth" "pose_encoder" "pose"

In [60]:
!pip -q install 'git+https://github.com/facebookresearch/segment-anything.git'
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

  Preparing metadata (setup.py) ... done
--2023-09-07 16:32:03--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.38, 13.35.7.128, 13.35.7.82, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  39.8MB/s    in 5.6s    

2023-09-07 16:32:08 (64.1 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]



In [61]:
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from PIL import Image
import numpy as np

sam_checkpoint = "/content/sam_vit_b_01ec64.pth"
model_type = "vit_b"

# device = "cuda"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)
sam.eval()
#if you want to change the size:
# sam, img_embedding_size = sam_model_registry[model_type](image_size=img_size,
#                                     num_classes=num_classes,
#                                     checkpoint=sam_checkpoint, pixel_mean=[0, 0, 0],
#                                     pixel_std=[1, 1, 1])

mask_generator = SamAutomaticMaskGenerator(sam)

# raw_img = Image.open('/content/raw_img.jpg')
# image = np.array(raw_img)
# masks = mask_generator.generate(image)
# print(masks[0].keys())

In [62]:
print(mask_generator)

In [63]:
sam_encoder = sam.image_encoder

In [ ]:
# input: torch.Size([3, 3, 256, 320])
# features: torch.Size([3, 48, 64, 80]) torch.Size([3, 80, 32, 40]) torch.Size([3, 128, 16, 20])
# output: torch.Size([3, 1, 256, 320])

In [75]:
!python thesis/train_end_to_end.py --no_cuda --height 224 --width 224 --scales 0 1 2 --data_path frames/d3k1 --log_dir sam  --batch_size 3 #--load_weights_folder womin/mdp/models/weights_19  --models_to_load "encoder" "depth" "pose_encoder" "pose"


/usr/local/lib/python3.10/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
[ 48  80 128] [0]
Training model named:
   mdp
Models and tensorboard events files are saved to:
   sam
Training is using:
   cpu
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoad

In [ ]:
# ["encoder = networks.ResnetEncoder(18, False)"]
# ["encoder = networks.LiteMono(model="lite-mono", width=feed_width, height=feed_height)"]

# depth_decoder = networks.DepthDecoder(num_ch_enc=encoder.num_ch_enc, scales=range(4))

# scales=range(4)

In [ ]:
!python thesis/test_simple.py --model_path sam/mdp/models/weights_18 --image_path SCARED --no_cuda --ext png

In [ ]:
!python thesis/export_gt_depth.py

In [ ]:
!python thesis/evaluate_depth.py --eval_mono --data_path frames/d3k1 --eval_split endovis --load_weights_folder sam/mdp/models/weights_18

# import numpy as np
# gt_depths = np.load("thesis/splits/endovis/gt_depths.npz")
# gt_depths = np.load("thesis/splits/endovis/gt_depths.npz", fix_imports=True, encoding='latin1')["data"]

In [ ]:
# class LGFI(nn.Module):
#     """
#     Local-Global Features Interaction
#     """
#     def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6, expan_ratio=6,
#                  use_pos_emb=True, num_heads=6, qkv_bias=True, attn_drop=0., drop=0.):
#         super().__init__()
#         self.swintran = create_model("swin_base_patch4_window7_224", pretrained=True)
#         # self.swintran.eval()
#         # for param in self.swintran.parameters():
#         #     param.requires_grad = False

#     def forward(self, x):
#       print("begining swin x:", x.size())

#       x = self.swintran.patch_embed(x)
#       x = self.swintran.layers(x)
#       x = self.swintran.norm(x).permute(0,3,1,2)
#       print("end swin x:", x.size())

#       return x